In [6]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
import pickle as pkl
import hist
import matplotlib.pyplot as plt
from analyzer.run_analysis import runModulesOnSamples
from analyzer.datasets import loadSamplesFromDirectory
import numpy as np
from analyzer.core import modules as all_modules
from analyzer.plotting.core_plots import *
from analyzer.plotting.simple_plot import Plotter
import math
import torch
import gpytorch
from matplotlib import pyplot as plt
loadStyles()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import pickle as pkl
file_name="../analyzerout/chargino_reco.pkl"
data = pkl.load(open(file_name, 'rb'))
histos = data["histograms"]

In [8]:
m4 = histos["m14_vs_m24"]["Skim_QCDInclusive2018",...]
m4

Hist(
  Regular(60, 0, 3000, name='$m_{4}$1', label='$m_{4}$'),
  Regular(60, 0, 3000, name='$m_{3}$2', label='$m_{3}$'),
  storage=Weight()) # Sum: WeightedSum(value=3.22924e+06, variance=4.26824e+07) (WeightedSum(value=3.29811e+06, variance=4.30507e+07) with flow)

In [9]:
bound = 1100
m4_real = m4
x,y = [x.centers for x in m4_real.axes]
X,Y = np.meshgrid(x,y)
points = np.stack((X.ravel(),Y.ravel()), axis=1)
print(points)
train_x = torch.from_numpy(points)
train_y = torch.from_numpy(m4_real.values().ravel())
print(train_y)
#train_y = train_y - train_y.mean(dim=-1,keepdim=True)

# We will use the simplest form of GP model, exact inference
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.RBFKernel(ard_num_dims=2) 
         #   gpytorch.kernels.RBFKernel(active_dims=torch.tensor([1,0])) * 
          #  gpytorch.kernels.RBFKernel(active_dims=torch.tensor([0,1])) * 
          #  gpytorch.kernels.RBFKernel(active_dims=torch.tensor([1,1])) 
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)
# initialize likelihood and model
v = torch.from_numpy(m4_real.variances())
likelihood = gpytorch.likelihoods.FixedNoiseGaussianLikelihood(noise=v, learn_additional_noise=True)
v = torch.sqrt(v) 
model = ExactGPModel(train_x, train_y, likelihood)
print(model)

[[  25.   25.]
 [  75.   25.]
 [ 125.   25.]
 ...
 [2875. 2975.]
 [2925. 2975.]
 [2975. 2975.]]
tensor([0., 0., 0.,  ..., 0., 0., 0.], dtype=torch.float64)
ExactGPModel(
  (likelihood): FixedNoiseGaussianLikelihood(
    (noise_covar): FixedGaussianNoise()
    (second_noise_covar): HomoskedasticNoise(
      (raw_noise_constraint): GreaterThan(1.000E-04)
    )
  )
  (mean_module): ConstantMean()
  (covar_module): ScaleKernel(
    (base_kernel): RBFKernel(
      (raw_lengthscale_constraint): Positive()
    )
    (raw_outputscale_constraint): Positive()
  )
)


/uscms_data/d3/ckapsiak/Analysis/SingleStop/SingleStopCoffea/env/lib/python3.9/site-packages/gpytorch/likelihoods/noise_models.py:148: NumericalWarning: Very small noise values detected. This will likely lead to numerical instabilities. Rounding small noise values up to 1e-06.
  warnings.warn(


In [ ]:
# this is for running the notebook in our testing framework
model = ExactGPModel(train_x, train_y, likelihood)
for param_name, param in model.named_parameters():
    print(f'Parameter name: {param_name:42} value = {param}')
    
#model.covar_module.base_kernel.lengthscale = 205
model.covar_module.outputscale = 10
model.covar_module.base_kernel.lengthscale=torch.tensor([600,600])
model.train()

likelihood.train()
optimizer = torch.optim.Adam([{"params": model.covar_module.base_kernel.parameters(), "lr": 25},
                              {"params": [model.covar_module.raw_outputscale], "lr": 1e9},
                              {"params": model.mean_module.parameters(), "lr": 500}
                            ]
                           )
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

for i in range(100):
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    output = model(train_x)
    # Calc loss and backprop gradients
    loss = -mll(output, train_y)
    loss.backward()
    for param_name, param in model.named_parameters():
        print(f'Parameter name: {param_name:42} value = {param}')
    #print('Iter %d - Loss: %.3f   lengthscale: %.3f  scale: %.3f  mean:%0.3f'  % (
      # i + 1,  loss.item(),
      #  model.covar_module.base_kernel..item(),
     #   model.covar_module.outputscale.item(),
      #  model.mean_module.constant.item()
   # ))
    optimizer.step()

Parameter name: likelihood.second_noise_covar.raw_noise    value = Parameter containing:
tensor([0.], requires_grad=True)
Parameter name: mean_module.raw_constant                   value = Parameter containing:
tensor(0., requires_grad=True)
Parameter name: covar_module.raw_outputscale               value = Parameter containing:
tensor(0., requires_grad=True)
Parameter name: covar_module.base_kernel.raw_lengthscale   value = Parameter containing:
tensor([[0., 0.]], requires_grad=True)
Parameter name: likelihood.second_noise_covar.raw_noise    value = Parameter containing:
tensor([0.], requires_grad=True)
Parameter name: mean_module.raw_constant                   value = Parameter containing:
tensor(0., requires_grad=True)
Parameter name: covar_module.raw_outputscale               value = Parameter containing:
tensor(10.0000, requires_grad=True)
Parameter name: covar_module.base_kernel.raw_lengthscale   value = Parameter containing:
tensor([[600., 600.]], requires_grad=True)


/uscms_data/d3/ckapsiak/Analysis/SingleStop/SingleStopCoffea/env/lib/python3.9/site-packages/linear_operator/utils/linear_cg.py:338: NumericalWarning: CG terminated in 1000 iterations with average residual norm 2.965822527889842 which is larger than the tolerance of 1 specified by linear_operator.settings.cg_tolerance. If performance is affected, consider raising the maximum number of CG iterations by running code in a linear_operator.settings.max_cg_iterations(value) context.
  warnings.warn(


Parameter name: likelihood.second_noise_covar.raw_noise    value = Parameter containing:
tensor([0.], requires_grad=True)
Parameter name: mean_module.raw_constant                   value = Parameter containing:
tensor(-500.0000, requires_grad=True)
Parameter name: covar_module.raw_outputscale               value = Parameter containing:
tensor(1.0000e+09, requires_grad=True)
Parameter name: covar_module.base_kernel.raw_lengthscale   value = Parameter containing:
tensor([[575., 575.]], requires_grad=True)


/uscms_data/d3/ckapsiak/Analysis/SingleStop/SingleStopCoffea/env/lib/python3.9/site-packages/linear_operator/utils/linear_cg.py:338: NumericalWarning: CG terminated in 1000 iterations with average residual norm 4.870234616010685 which is larger than the tolerance of 1 specified by linear_operator.settings.cg_tolerance. If performance is affected, consider raising the maximum number of CG iterations by running code in a linear_operator.settings.max_cg_iterations(value) context.
  warnings.warn(


Parameter name: likelihood.second_noise_covar.raw_noise    value = Parameter containing:
tensor([0.], requires_grad=True)
Parameter name: mean_module.raw_constant                   value = Parameter containing:
tensor(-835.0302, requires_grad=True)
Parameter name: covar_module.raw_outputscale               value = Parameter containing:
tensor(1.6701e+09, requires_grad=True)
Parameter name: covar_module.base_kernel.raw_lengthscale   value = Parameter containing:
tensor([[557.9842, 557.8680]], requires_grad=True)


...

Parameter name: likelihood.second_noise_covar.raw_noise    value = Parameter containing:
tensor([0.], requires_grad=True)
Parameter name: mean_module.raw_constant                   value = Parameter containing:
tensor(-1094.0100, requires_grad=True)
Parameter name: covar_module.raw_outputscale               value = Parameter containing:
tensor(2.1880e+09, requires_grad=True)
Parameter name: covar_module.base_kernel.raw_lengthscale   value = Parameter containing:
tensor([[544.7482, 544.5128]], requires_grad=True)


/uscms_data/d3/ckapsiak/Analysis/SingleStop/SingleStopCoffea/env/lib/python3.9/site-packages/linear_operator/utils/linear_cg.py:338: NumericalWarning: CG terminated in 1000 iterations with average residual norm 13.553492364315684 which is larger than the tolerance of 1 specified by linear_operator.settings.cg_tolerance. If performance is affected, consider raising the maximum number of CG iterations by running code in a linear_operator.settings.max_cg_iterations(value) context.
  warnings.warn(


Parameter name: likelihood.second_noise_covar.raw_noise    value = Parameter containing:
tensor([0.], requires_grad=True)
Parameter name: mean_module.raw_constant                   value = Parameter containing:
tensor(-1306.1500, requires_grad=True)
Parameter name: covar_module.raw_outputscale               value = Parameter containing:
tensor(2.6123e+09, requires_grad=True)
Parameter name: covar_module.base_kernel.raw_lengthscale   value = Parameter containing:
tensor([[533.8550, 533.5071]], requires_grad=True)


/uscms_data/d3/ckapsiak/Analysis/SingleStop/SingleStopCoffea/env/lib/python3.9/site-packages/linear_operator/utils/linear_cg.py:338: NumericalWarning: CG terminated in 1000 iterations with average residual norm 2.7445226957757978 which is larger than the tolerance of 1 specified by linear_operator.settings.cg_tolerance. If performance is affected, consider raising the maximum number of CG iterations by running code in a linear_operator.settings.max_cg_iterations(value) context.
  warnings.warn(


Parameter name: likelihood.second_noise_covar.raw_noise    value = Parameter containing:
tensor([0.], requires_grad=True)
Parameter name: mean_module.raw_constant                   value = Parameter containing:
tensor(-1485.4564, requires_grad=True)
Parameter name: covar_module.raw_outputscale               value = Parameter containing:
tensor(2.9709e+09, requires_grad=True)
Parameter name: covar_module.base_kernel.raw_lengthscale   value = Parameter containing:
tensor([[524.6142, 524.1560]], requires_grad=True)


In [ ]:
# Get into evaluation (predictive posterior) mode
model.eval()
likelihood.eval()
#model.covar_module.base_kernel.lengthscale = 10

# Test points are regularly spaced along [0,1]
# Make predictions by feeding model through likelihood
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    test_x = torch.linspace(1000, 3100,1500)
    #test_x = train_x.flatten()
    observed_pred = likelihood(model(test_x),noise=torch.zeros_like(test_x).flatten())
    #observed_pred = model(test_x)
    print(model.mean_module.constant)
    print(observed_pred.mean)
    # Initialize plot
    f, ax = plt.subplots()

    # Get upper and lower confidence bounds
    lower, upper = observed_pred.confidence_region()
    # Plot training data as black stars
    #ax.plot(train_x.numpy(), train_y.numpy(), 'k*')
    ax.errorbar(train_x.numpy(), train_y.numpy(), yerr=v, fmt='o', markersize=3)

    # Plot predictive means as blue line
    ax.plot(test_x.numpy(), observed_pred.mean.numpy(), 'b')
    # Shade between the lower and upper confidence bounds
    ax.fill_between(test_x.numpy(), lower.numpy(), upper.numpy(), alpha=0.5)

    ax.legend(['Observed Data', 'Mean', 'Confidence'])
